In [1]:
from __future__ import print_function, division
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os

chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)

In [177]:
# log in and go to page to get data
driver.get('https://fantasydata.com/user/login')
time.sleep(1);
logIn = driver.find_element_by_name('Email')
logIn.send_keys('rishis1096@gmail.com')
logIn = driver.find_element_by_name('Password')
logIn.send_keys('password') # false password due to code being put online
logIn.send_keys(Keys.RETURN)

In [178]:
driver.get('https://fantasydata.com/nfl-stats/fantasy-football-leaders?position=2&team=1&season=2017&seasontype=1&scope=2&subscope=1&startweek=1&endweek=17')
time.sleep(1);

In [183]:
# use beautifulsoup to scrape - starting with player name
innerHTML = driver.execute_script("return document.body.innerHTML")
innerHTML = BeautifulSoup(innerHTML, 'lxml')
innerHTML

<html><head><style>
    .player-search-combo {
        border-color: #eceef0 !important;
    }

    .player-search-combo .k-multiselect-wrap {
        display: flex;
        flex-wrap: wrap;
        padding-right: 5px;
    }

    .player-search-combo .k-multiselect-wrap .k-input {
        order: 1;
        width: 100% !important;
    }

    .player-search-combo .k-multiselect-wrap .k-input {
        width: 100% !important;
    }

    .player-search-combo .k-multiselect-wrap .k-reset {
        width: 100%;
    }

    .player-search-combo .k-multiselect-wrap .k-reset li {
        width: 100%;
        height: 31px;
        border-radius: 4px;
        border-color: #ccc;
    }

    .player-search-combo .show-icon {
        display: block !important;
    }

    .player-search-combo .k-clear-value {
        top: unset !important;
        bottom: 10px;
    }

    /*.k-multiselect .k-hidden {
        display: block !important;
    }*/

    ::placeholder { /* Chrome, Firefox, Opera, Safari 10.1

In [186]:
# get the names of the QBs in order
names = [str(link) for link in innerHTML.find_all('a') if 'nfl-stats/player-details' in str(link)]
names = [link.split('>')[1].split('<')[0] for link in names]
names

['Matt Ryan',
 'Matt Ryan',
 'Matt Ryan',
 'Matt Ryan',
 'Matt Ryan',
 'Matt Ryan',
 'Matt Ryan',
 'Matt Ryan',
 'Matt Ryan',
 'Matt Ryan',
 'Matt Ryan',
 'Matt Ryan',
 'Matt Ryan',
 'Matt Ryan',
 'Matt Ryan',
 'Matt Ryan']

In [181]:
# get week numbers in order
weeks = [str(week) for week in innerHTML.find_all('span') if 'dataItem.Week' in str(week)]
weeks = [week.split('>')[1].split('<')[0] for week in weeks]
weeks

['9',
 '17',
 '1',
 '7',
 '11',
 '12',
 '8',
 '15',
 '2',
 '10',
 '16',
 '3',
 '6',
 '4',
 '13',
 '14']

In [185]:
# get opponents in order
opponents = [str(opponent) for opponent in innerHTML.find_all('span') if 'dataItem.Opponent' in str(opponent)]
opponents = [opponent.split('>')[1].split('<')[0] for opponent in opponents]
opponents

['CAR',
 'CAR',
 'CHI',
 'NE',
 'SEA',
 'TB',
 'NYJ',
 'TB',
 'GB',
 'DAL',
 'NO',
 'DET',
 'MIA',
 'BUF',
 'MIN',
 'NO']

In [189]:
# get data in order
data = [str(data) for data in innerHTML.find_all('td') if 'class=""' in str(data) and 'span' not in str(data)]
data = [data for data in data if 'href' not in data]
data = [data.split('>')[1].split('<')[0] for data in data]
newData = []
for x in range(len(data)//12):
    newData.append(data[12*x:12*x + 11])
for statList in newData:
    if len(statList) == 11:
        statList.append(0)

completions = []
attempts = []
percentage = []
passingYards = []
avgYdsThrow = []
passTDs = []
interceptions = []
passerRating = []
rushAtt = []
rushYds = []
rushAvg = []
rushTDs = []

for x in range(len(newData)):
    completions.append(newData[x][0])
    attempts.append(newData[x][1])
    percentage.append(newData[x][2])
    passingYards.append(newData[x][3])
    avgYdsThrow.append(newData[x][4])
    passTDs.append(newData[x][5])
    interceptions.append(newData[x][6])
    passerRating.append(newData[x][7])
    rushAtt.append(newData[x][8])
    rushYds.append(newData[x][9])
    rushAvg.append(newData[x][10])
    rushTDs.append(newData[x][11])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [190]:
# get fantasy points in order
fantasyPoints = [str(fpts) for fpts in innerHTML.find_all('span') if 'dataItem.FantasyPoints' in str(fpts) and 'PerGame' not in str(fpts)]
fantasyPoints = [fpts.split('>')[1].split('<')[0] for fpts in fantasyPoints]
fantasyPoints

['18.42',
 '18.08',
 '17.94',
 '17.02',
 '17',
 '16.78',
 '15.86',
 '15.38',
 '14.88',
 '14.6',
 '14.22',
 '13.66',
 '11.92',
 '8.48',
 '7.02',
 '6.84']

In [191]:
ATL = pd.DataFrame({'Name':names, 'Week':weeks, 'OPP':opponents, 'CMP':completions, 'ATT':attempts, 'PCT':percentage, 'PassYd':passingYards, 'AvgYds/Pass':avgYdsThrow, 'PassTD':passTDs, 'INT':interceptions, 'Rating':passerRating, 'RushAtt':rushAtt, 'RushYds':rushYds, 'AvgYds/Rush':rushAvg, 'RushTD':rushTDs, 'Fantasy Points':fantasyPoints})
ATL

,Name,Week,OPP,CMP,ATT,PCT,PassYd,AvgYds/Pass,PassTD,INT,Rating,RushAtt,RushYds,AvgYds/Rush,RushTD,Fantasy Points
0,Matt Ryan,9,CAR,24,38,63.2,313,8.2,2,1,95.61,1,-1,-1,0,18.42
1,Matt Ryan,17,CAR,28,45,62.2,317,7,1,0,90.69,3,14,4.7,0,18.08
2,Matt Ryan,1,CHI,21,30,70,321,10.7,1,0,116.11,3,11,3.7,0,17.94
3,Matt Ryan,7,NE,23,33,69.7,233,7.1,1,0,99.68,3,37,12.3,0,17.02
4,Matt Ryan,11,SEA,19,27,70.4,195,7.2,2,0,115.51,3,12,4,0,17
5,Matt Ryan,12,TB,26,35,74.3,317,9.1,1,0,111.25,1,1,1,0,16.78
6,Matt Ryan,8,NYJ,18,29,62.1,254,8.8,2,0,113.29,6,17,2.8,0,15.86
7,Matt Ryan,15,TB,17,31,54.8,212,6.8,1,0,87.03,3,29,9.7,0,15.38
8,Matt Ryan,2,GB,19,28,67.9,252,9,1,0,108.04,1,8,8,0,14.88
9,Matt Ryan,10,DAL,22,29,75.9,215,7.4,2,1,104.81,0,0,0,0,14.6
